In [160]:
!pip install twython

In [161]:
from twython import Twython
from credentials import *
from OAUTH import *
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET_KEY,OAUTH_TOKEN,OAUTH_TOKEN_SECRET)
import re

In [162]:
results = twitter.cursor(twitter.search,q ='#savetheplanet', lang = 'en', result_type = 'mixed', count = 500, tweet_mode = 'extended')

In [163]:
data = []
for result in results:
    data.append(result)

In [164]:
import pandas as pd

In [165]:
df = pd.json_normalize(data)

In [166]:
df.head(2)

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,quoted_status.place.id,quoted_status.place.url,quoted_status.place.place_type,quoted_status.place.name,quoted_status.place.full_name,quoted_status.place.country_code,quoted_status.place.country,quoted_status.place.contained_within,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates
0,Wed Nov 24 11:08:47 +0000 2021,1463464609147498507,1463464609147498507,Beautiful 💝💝 #FaisalMasjid #Islamabad but visi...,False,"[0, 180]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Nov 28 18:41:24 +0000 2021,1465028067705999360,1465028067705999360,RT @ClubTrp: O dear humans! We are on the brin...,False,"[0, 140]","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
df1 = df.iloc[:, [0, 1,3,6,35,41,42,45]]

In [168]:
df1['created_at'] = pd.to_datetime(df1['created_at'])
df1.head(2)

<ipython-input-168-1b26ef118de4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['created_at'] = pd.to_datetime(df1['created_at'])


,created_at,id,full_text,source,user.location,user.followers_count,user.friends_count,user.favourites_count
0,2021-11-24 11:08:47+00:00,1463464609147498507,Beautiful 💝💝 #FaisalMasjid #Islamabad but visi...,"<a href=""http://twitter.com/download/android"" ...","Islamabad, Pakistan",77752,2147,118675
1,2021-11-28 18:41:24+00:00,1465028067705999360,RT @ClubTrp: O dear humans! We are on the brin...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Climate Live,246,281,2502


In [169]:
df1.columns = ['created_at', 'user_id', 'tweet', 'device_type', 'location', 'followers_count', 'friends_count', 'favourites_count']

In [170]:
df1.head(2)

,created_at,user_id,tweet,device_type,location,followers_count,friends_count,favourites_count
0,2021-11-24 11:08:47+00:00,1463464609147498507,Beautiful 💝💝 #FaisalMasjid #Islamabad but visi...,"<a href=""http://twitter.com/download/android"" ...","Islamabad, Pakistan",77752,2147,118675
1,2021-11-28 18:41:24+00:00,1465028067705999360,RT @ClubTrp: O dear humans! We are on the brin...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Climate Live,246,281,2502


In [171]:
df1['device_type'][0]

'<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>'

In [172]:
df1['device_type']= df1['device_type'].str.extract(r'(\w+\s\w+\s\w+)')

<ipython-input-172-360baa7a0ee2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['device_type']= df1['device_type'].str.extract(r'(\w+\s\w+\s\w+)')


In [173]:
df1['device_type'] = df1['device_type'].str.extract(r'(A\w+|iP\w+|W\w+ A\w+|Mac|Bot|bot)')

<ipython-input-173-8e47cf1646bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['device_type'] = df1['device_type'].str.extract(r'(A\w+|iP\w+|W\w+ A\w+|Mac|Bot|bot)')


In [176]:
df1['device_type'] = df1['device_type'].str.replace('bot', 'Bot')

<ipython-input-176-d3eb7791ed34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['device_type'] = df1['device_type'].str.replace('bot', 'Bot')


In [178]:
df1['Analysis'] = df1['tweet']

<ipython-input-178-f628ddabf5bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Analysis'] = df1['tweet']


In [183]:
df1['Analysis'][0]

'Beautiful 💝💝 #FaisalMasjid #Islamabad but visitors are ruining its beauty with throwing trash everywhere...\n\n#environmentawareness #savetheplanet #StopLittering #KeepIslamabadClean https://t.co/tsG6tiT77i'

In [186]:
df1['Analysis'] = df1['Analysis'].str.lower()

<ipython-input-186-5400851e40eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Analysis'] = df1['Analysis'].str.lower()


In [193]:
df1['Analysis'] = df1['Analysis'].str.replace(pat =r'\W+', repl = ' ', regex = True)

<ipython-input-193-b37436583bf6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Analysis'] = df1['Analysis'].str.replace(pat =r'\W+', repl = ' ', regex = True)


'beautiful faisalmasjid islamabad but visitors are ruining its beauty with throwing trash everywhere environmentawareness savetheplanet stoplittering keepislamabadclean https t co tsg6tit77i'

In [197]:
df1 ['Analysis'] = df1['Analysis'].str.replace(pat = r'\d+', repl = ' ', regex= True)

<ipython-input-197-2cd036b1207d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1 ['Analysis'] = df1['Analysis'].str.replace(pat = r'\d+', repl = ' ', regex= True)


In [202]:
df1['Analysis'][0]

'beautiful faisalmasjid islamabad but visitors are ruining its beauty with throwing trash everywhere environmentawareness savetheplanet stoplittering keepislamabadclean https t co tsg tit i'

In [200]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [201]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [212]:
all_stopwords = stopwords.words('english')
all_stopwords.extend(['ma','this', 'https','tit', 'tsg', 'co', 'rt'])

In [204]:
testing_ = df1['Analysis']

In [206]:
testing_

0       beautiful faisalmasjid islamabad but visitors ...
1       rt clubtrp o dear humans we are on the brink o...
2       check out our ecofriendly phone cases   biodeg...
3       some lovely alternatives to sending the tradit...
4       rt clubtrp o dear humans we are on the brink o...
                              ...                        
3236                                                  NaN
3237                                                  NaN
3238                                                  NaN
3239                                                  NaN
3240                                                  NaN
Name: Analysis, Length: 3241, dtype: object

In [213]:
# tokenizing the words
test_ = []
for i in testing_:
   try:
    text_tokens = word_tokenize(i, language = "english")
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    test_.append(tokens_without_sw)
   except TypeError:
       test_.append('None')
test_[0]

['beautiful',
 'faisalmasjid',
 'islamabad',
 'visitors',
 'ruining',
 'beauty',
 'throwing',
 'trash',
 'everywhere',
 'environmentawareness',
 'savetheplanet',
 'stoplittering',
 'keepislamabadclean']

In [214]:
# now combine all the words into a single sentence by using the 'join' function

for i in range(len(test_)):
    test_[i] = " ".join(test_[i]) #here this part of the code will join all the words together

df1['Analysis'] = test_

<ipython-input-214-efe4299f184e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Analysis'] = test_


In [216]:
df1.head(2)

,created_at,user_id,tweet,device_type,location,followers_count,friends_count,favourites_count,Analysis
0,2021-11-24 11:08:47+00:00,1463464609147498507,Beautiful 💝💝 #FaisalMasjid #Islamabad but visi...,Android,"Islamabad, Pakistan",77752,2147,118675,beautiful faisalmasjid islamabad visitors ruin...
1,2021-11-28 18:41:24+00:00,1465028067705999360,RT @ClubTrp: O dear humans! We are on the brin...,Web App,Climate Live,246,281,2502,clubtrp dear humans brink boiling point right ...


## __Sentiment Analysis__

- ### The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity).
- ### The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0]
- ### where 0.0 is very objective (non-friendly) and 1.0 is very subjective (normal or friendly).

## __textblob__

- ### It is commonly used library in NLP (natural language processing) to identify the subjective of the matter
- ### [link for explaination]('https://www.analyticsvidhya.com/blog/2018/02/natural-language-processing-for-beginners-using-textblob/')
- ### [link for library]('https://textblob.readthedocs.io/en/dev/quickstart.html')
- ### [What is polarity and subjectivity in sentiment analysis?]('https://www.quora.com/What-is-polarity-and-subjectivity-in-sentiment-analysis')
    - ### Polarity: It simply means emotions expressed in a sentence. Emotions are closely related to sentiments. The strength of a sentiment or opinion is typically linked to the intensity of certain emotions, e.g., joy and anger. Opinions in sentiment analysis are mostly evaluations(although not always).
    - ### Polarity example: For example, the following sentences express emotional evaluations: “I love iPhone,” “I am so angry with their service people” and “This is the best car ever built.” To make use of these two types of evaluations in practice, we can design 5 sentiment ratings, emotional negative (-2), rational negative (-1), neutral (0), rational positive (+1), and emotional positive (+2). In practice, neutral often means no opinion or sentiment expressed.
    - ### Subjectivity: Subjective sentence expresses some personal feelings, views, or beliefs. For an example:- subjective sentence is “I like iPhone.” Subjective expressions come in many forms, e.g., opinions, allegations, desires, beliefs, suspicions, and speculations. For example, “I think that he went home” and “I want a camera that can take good photos” are a subjective sentences, but does not express any sentiment.

In [217]:
!pip install -U textblob

In [220]:
df1['Analysis'][0]

'beautiful faisalmasjid islamabad visitors ruining beauty throwing trash everywhere environmentawareness savetheplanet stoplittering keepislamabadclean'

In [224]:
anay_ = df1['Analysis']

In [244]:
from textblob import TextBlob
subjectivity_score = []
polarity_score = []
for i in anay_:
    testimonial = TextBlob(i)
    subjectivity_score.append(round(testimonial.sentiment.subjectivity))
    polarity_score.append(round(testimonial.sentiment.polarity))

In [246]:
df1['subjectivity_score'] = subjectivity_score

<ipython-input-246-b79a5752d534>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['subjectivity_score'] = subjectivity_score


In [247]:
df1['subjectivity_score'].value_counts()

0    3194
1      47
Name: subjectivity_score, dtype: int64

In [275]:
df1['Analysis'] = df1['Analysis'].str.replace(pat = 'N n e', repl = 'NaN')

<ipython-input-275-6f65d3e5c059>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Analysis'] = df1['Analysis'].str.replace(pat = 'N n e', repl = 'NaN')


In [279]:
df1.dropna(thresh = 9)

,created_at,user_id,tweet,device_type,location,followers_count,friends_count,favourites_count,Analysis,subjectivity_score
0,2021-11-24 11:08:47+00:00,1463464609147498507,Beautiful 💝💝 #FaisalMasjid #Islamabad but visi...,Android,"Islamabad, Pakistan",77752,2147,118675,beautiful faisalmasjid islamabad visitors ruin...,1
1,2021-11-28 18:41:24+00:00,1465028067705999360,RT @ClubTrp: O dear humans! We are on the brin...,Web App,Climate Live,246,281,2502,clubtrp dear humans brink boiling point right ...,1
2,2021-11-28 18:36:45+00:00,1465026896014553088,Check out our #ecofriendly phone cases! 100% b...,iPhone,,99,616,42,check ecofriendly phone cases biodegradable de...,1
3,2021-11-28 18:36:06+00:00,1465026730360549380,Some lovely alternatives to sending the tradit...,iPhone,,1018,989,9891,lovely alternatives sending traditional christ...,1
4,2021-11-28 18:35:23+00:00,1465026551368531976,RT @ClubTrp: O dear humans! We are on the brin...,Web App,Brasil,1214,1646,30310,clubtrp dear humans brink boiling point right ...,1
...,...,...,...,...,...,...,...,...,...,...
3235,2021-11-20 01:12:39+00:00,1461865034796220417,NaN,Bot,Earth 🌏,1482,1,146,NaN,0
3236,2021-11-20 01:12:33+00:00,1461865013275238403,NaN,Bot,Earth 🌏,1482,1,146,NaN,0
3237,2021-11-20 01:12:28+00:00,1461864991745789954,NaN,Bot,Earth 🌏,1482,1,146,NaN,0
3238,2021-11-20 01:03:17+00:00,1461862678054551554,NaN,iPhone,,254,263,8072,NaN,0


In [61]:
for i in df.columns:
    print(i)

created_at
id
id_str
full_text
truncated
display_text_range
source
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
geo
coordinates
place
contributors
is_quote_status
retweet_count
favorite_count
favorited
retweeted
possibly_sensitive
lang
entities.hashtags
entities.symbols
entities.user_mentions
entities.urls
entities.media
extended_entities.media
metadata.result_type
metadata.iso_language_code
user.id
user.id_str
user.name
user.screen_name
user.location
user.description
user.url
user.entities.url.urls
user.entities.description.urls
user.protected
user.followers_count
user.friends_count
user.listed_count
user.created_at
user.favourites_count
user.utc_offset
user.time_zone
user.geo_enabled
user.verified
user.statuses_count
user.lang
user.contributors_enabled
user.is_translator
user.is_translation_enabled
user.profile_background_color
user.profile_background_image_url
user.profile_background_image_url_https
user.profile_